In [55]:
# import packages

import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', None)

In [56]:
link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTk6t3NoeAIDC82RnfQO9As7eyuvVRx-Y8uwHoeJqefuXNvMhiyFdD0TizGwu1ktsnOeWGm9J0LahpR/pub?output=csv'
strava_data = pd.read_csv(link, header=0)
runs_df = strava_data.loc[strava_data['type'] == 'Run']
runs_df.head()

,name,distance_m,moving_time_s,elapsed_time_s,elevation_change_m,type,date,average_cadence,average_heartrate,max_heartrate,pace_per_mile,distance_mi,id
0,Evening Run,5823,1825,1828,10.9,Run,12/26/2022,86.7,152.4,167,8:24,3.62,NaN
1,Morning Run,2650,1110,1633,-6.6,Run,12/28/2022,73.7,133.0,157,11:13,1.65,NaN
2,Evening Run,2813,896,896,23.2,Run,1/8/2023,86.5,160.9,173,8:32,1.75,NaN
3,Afternoon Run,3382,1102,1179,18.9,Run,2/4/2023,85.9,141.4,176,8:44,2.10,NaN
4,Afternoon Run,3652,1355,1357,31.8,Run,2/6/2023,82.7,146.8,166,9:56,2.27,NaN


In [57]:
runs_df['date'] = pd.to_datetime(runs_df['date'], format='mixed').dt.date
runs_df['moving_time_s'] = pd.to_timedelta(runs_df['moving_time_s'], unit='s')
runs_df['pace_per_mile'] = runs_df['moving_time_s'] / runs_df['distance_mi']
runs_df.sort_values(by=['date']).tail(10)

,name,distance_m,moving_time_s,elapsed_time_s,elevation_change_m,type,date,average_cadence,average_heartrate,max_heartrate,pace_per_mile,distance_mi,id
162,Morning Run,8050,0 days 00:44:03,2643,-43.5,Run,2024-06-02,84.8,159.2,177,0 days 00:08:48.600000,5.00,NaN
163,Evening Run,2811,0 days 00:15:54,981,-73.0,Run,2024-06-04,84.3,146.3,160,0 days 00:09:05.142857142,1.75,NaN
164,Evening Run,4819,0 days 00:30:40,1957,-28.4,Run,2024-06-06,83.1,135.8,145,0 days 00:10:13.333333333,3.00,NaN
165,Morning Run,3287,0 days 00:14:55,1527,-15.6,Run,2024-06-08,88.8,152.0,181,0 days 00:07:18.725490196,2.04,NaN
166,Morning Run,10027,0 days 00:54:53,3307,19.2,Run,2024-06-09,85.5,156.9,170,0 days 00:08:48.571428571,6.23,NaN
167,Morning Run,3993,0 days 00:20:26,1229,0.0,Run,2024-06-16,83.2,139.2,154,0 days 00:08:14.354838709,2.48,NaN
168,Evening Run,2445,0 days 00:15:08,930,7.1,Run,2024-06-18,82.6,131.6,147,0 days 00:09:57.368421052,1.52,NaN
169,Evening Run,4921,0 days 00:30:18,1831,17.1,Run,2024-06-20,83.0,150.2,164,0 days 00:09:54.117647058,3.06,NaN
170,Evening Run,5989,0 days 00:35:53,2170,16.5,Run,2024-06-27,82.8,137.0,151,0 days 00:09:38.763440860,3.72,1.175660e+10
171,Morning Run,4392,0 days 00:24:19,1499,16.6,Run,2024-06-29,83.5,143.5,157,0 days 00:08:54.432234432,2.73,1.176999e+10


In [58]:
runs_df.isna().sum()

name                    0
distance_m              0
moving_time_s           0
elapsed_time_s          0
elevation_change_m      0
type                    0
date                    0
average_cadence         0
average_heartrate       0
max_heartrate           0
pace_per_mile           0
distance_mi             0
id                    170
dtype: int64

In [59]:
# convert column names to lower case and snake case
column_names = {column: column.strip().lower().replace(' ', '_').replace('.1', '') for column in runs_df.columns}
runs_df = runs_df.rename(columns=column_names)

In [60]:
# convert name to a categorical for time of day for the run
runs_df['time_of_day'] = runs_df['name'].str.lower().str.split(' ').str[0]
runs_df['time_of_day'] = runs_df['time_of_day'].replace({'lunch': 'afternoon'}).astype('category')
runs_df['time_of_day']

0        evening
1        morning
2        evening
3      afternoon
4      afternoon
         ...    
167      morning
168      evening
169      evening
170      evening
171      morning
Name: time_of_day, Length: 172, dtype: category
Categories (4, object): ['afternoon', 'evening', 'morning', 'night']

In [61]:
# determine average running zone based on averagee heart rate

AGE = 29
MAX_HEART_RATE = 220 - AGE

def find_zone(avg_heart_rate):
    if avg_heart_rate < 0.6 * MAX_HEART_RATE:
        return 1
    elif avg_heart_rate < 0.7 * MAX_HEART_RATE:
        return 2
    elif avg_heart_rate < 0.8 * MAX_HEART_RATE:
        return 3
    elif avg_heart_rate < 0.9 * MAX_HEART_RATE:
        return 4
    else:
        return 5

In [62]:
runs_df['average_zone'] = runs_df['average_heartrate'].apply(find_zone)
runs_df['max_zone']  = runs_df['max_heartrate'].apply(find_zone)
runs_df['ratio_avg_hr_to_max_hr'] = runs_df['average_heartrate'] / runs_df['max_heartrate']

In [63]:
runs_df.to_csv('run_data.csv', index=False)

In [64]:
def load_df():
    return runs_df